### Didnt need to do all this since I believe I really only needed fd_list.csv, but I was just testing things out.

In [ ]:
import os
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from PIL import Image
import cv2
from math import sqrt
import time

from skimage.io import imread, imshow, imread_collection
from skimage.transform import resize
from skimage.feature import hog
from skimage import exposure

from sklearn.model_selection import cross_val_score
from sklearn.feature_selection import SelectKBest, chi2, RFE, RFECV
from sklearn.tree import DecisionTreeClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.svm import SVC
import pickle

from google.colab import drive

In [ ]:
drive.mount('/content/drive', force_remount=True)

train_img = '/content/drive/Shareddrives/CSCE 633(Fall 2020) HW5/train_resized_100'
train_csv = '/content/drive/Shareddrives/CSCE 633(Fall 2020) HW5/train.csv'
test_img = '/content/drive/Shareddrives/CSCE 633(Fall 2020) HW5/test_resized_100'
test_csv = '/content/drive/Shareddrives/CSCE 633(Fall 2020) HW5/test.csv'

In [ ]:
train = pd.read_csv(train_csv)
test = pd.read_csv(test_csv)

In [ ]:
out_val = train['covid(label)'].value_counts()
print('Covid-: %d\nCovid+: %d' % (out_val[0], out_val[1]))

In [ ]:
#  https://machinelearningmastery.com/how-to-manually-scale-image-pixel-data-for-deep-learning/
# Not scaling image size but data
def per_image_standardization(img):
  mean = img.mean()
  std = img.std()
  std = max(std, 1.0/sqrt(len(img)))
  img = (img - mean) / std
  # clip pixel values to [-1,1]
  img = np.clip(img, -1.0, 1.0)
  # shift from [-1,1] to [0,1] with 0.5 mean
  img = (img + 1.0) / 2.0
  return img
  
def Preprocessing(loc,data):
    image_list = []
    for image_name in data.filename:
        # img = imread(loc + image_name, as_gray=True)
        img = per_image_standardization(imread(loc +'/' +image_name, as_gray=True).astype("float32"))
        image_list.append(img)
    image_list = np.array(image_list)
    # image_list = image_list.astype("float32")
    # image_list = (image_list / 255) # - 0.5
    return image_list

images_train = Preprocessing(train_img, train)
images_test = Preprocessing(test_img, test)

In [ ]:
test.columns.values

In [ ]:
print(images_train[0][0])
imshow(images_train[0])
print(images_test[0][0])
imshow(images_test[0])

# REPLACE HERE WITH fd_list.csv (I believe) Actually just use the extract features part below since setup to work with current image size

In [ ]:
# testing it out with smaller images
def extract_features_hog(images):
  image_list = []
  fd_list = []
  for img in images:
    img = resize(img, (100,50))
    fd, hog_image = hog(img, feature_vector=True, visualize=True)
    hog_image = exposure.rescale_intensity(hog_image, in_range=(0, 10)) 
    fd_list.append(fd)
    image_list.append(hog_image)
  return np.array(image_list), np.array(fd_list)
hog_imgs, hog_fd = extract_features_hog(images_train)
hog_imgs_test, hog_fd_test = extract_features_hog(images_test)

In [ ]:
# X = hog_fd
# y = train['covid(label)']
# print(X[0].shape)

X_test = hog_fd_test
y_test = test['covid(label)']
print(X_test[0].shape)

In [ ]:
print(len(X))
print(type(y))
print(len(X[0]))
print(y[0])


In [ ]:
df = pd.DataFrame(data=X)
df['covid(label)']=y
print(df.columns.values)

In [ ]:
# df.to_csv('/content/drive/Shareddrives/CSCE 633(Fall 2020) HW5/hogged_data.csv', index=False)

# Wrapper code

Instructions: 
* use a simple classifier such as logister regression or svm
* plot the classification performance using a 5-fold cross-validation on
the training data against the number of features for both feature selection methods
* Compare
and contrast between the two (e.g., in terms of performance and computation time)

In [ ]:
# Instructions: use a simple classifier such as logister regression or svm
# different ways we can use for 
# est = SVC(kernel="linear", C=1)
# est = LogisticRegression(solver="lbfgs", max_iter=300)
# rfe = RFE(estimator=DecisionTreeClassifier(), 50)
# rfe = RFECV(estimator=est)
# rfe = RFE(estimator=est, verbose=1)

In [ ]:
# wrapper = rfe.fit_transform(X2, y)

In [ ]:
svc = SVC(kernel="linear")
rfecv = RFECV(estimator=svc, step=1, cv=5, scoring='accuracy', verbose=1)
# rfecv = RFECV(estimator=svc, step=1, cv=StratifiedKFold(5), scoring='accuracy', verbose=1)


**Skip next 2 steps & instead load model from file**


In [ ]:
bg = time.time()
rfecv.fit(X, y)
end = time.time()-bg
print("Time to Fit:", end)

In [ ]:
# run this if you reran rfecv & change filename
def save_model(model, name):
  Pkl_Filename = "/content/drive/Shareddrives/CSCE 633(Fall 2020) HW5/"+name+".pkl"
  with open(Pkl_Filename, 'wb') as file:  
    pickle.dump(model, file)

save_model(rfecv, 'RFEcv_model_2')

In [ ]:
# Load the Model back from file
def load_model(name):
  Pkl_Filename = '/content/drive/Shareddrives/CSCE 633(Fall 2020) HW5/'+name+'.pkl'
  with open(Pkl_Filename, 'rb') as file:  
      model = pickle.load(file)
  return model



In [ ]:
# Choose one
# svm model
rfecv = load_model('RFEcv_model')
# logistic regression model
rfecv_log = load_model('rfecv_log')

In [ ]:
print("Optimal number of features : %d" % rfecv.n_features_)
print("RFE_CV indices:", rfecv.get_support(indices = True))
print("Ranked Indices:", rfecv.ranking_)

In [ ]:
# get the selected data by using I believe.
selected_rfe = rfecv.transform(X)

In [ ]:
plt.figure()
plt.xlabel("Number of features selected")
plt.ylabel("Cross validation score (nb of correct classifications)")
plt.plot(range(1, len(rfecv.grid_scores_) + 1), rfecv.grid_scores_)
plt.show()

**Filter?**

In [ ]:
# k = # of features you want
bestfeatures_filter = SelectKBest(score_func=chi2, k=100)
filter_fit = bestfeatures_filter.fit(X,y)
print("Filter indices:", filter_fit.get_support(indices = True))

Timing:
* RFECV LogisticRegression:
 - 1388.21655249595
* RFECV SVC:
 - 2671.6101694107
* RFE Log:
 - 199.14080786705
* RFE SVC:
 - 399.396792173385


In [ ]:
def cross_val(X_val):
  score = []
  skf=KFold(n_splits=5, random_state=None, shuffle=True) 
  for train_index, test_index in skf.split(X_val, y):
    X_train, X_test = X_val[train_index], X_val[test_index]
    y_train, y_test = y[train_index], y[test_index]
    lg = LogisticRegression(solver='lbfgs', max_iter=500)
    lg.fit(X_train, y_train)
    score.append(lg.score(X_test, y_test))
  return score

In [ ]:
rfecv = load_model('RFEcv_model')
rfecv_log = load_model('rfecv_log')

In [ ]:
rf_score = cross_val(rfecv_log.transform(X))
rf_score2 = cross_val(rfecv.transform(X))
filter_score = cross_val(filter_fit.transform(X))
plt.plot(range(1, len(rf_score)+1), rf_score)
plt.plot(range(1, len(rf_score2)+1), rf_score2)
plt.plot(range(1, len(filter_score)+1), filter_score)
plt.yticks([0.5, 0.75, 1])
plt.xticks([1, 2, 3, 4, 5])

In [ ]:


rfecv = load_model('RFEcv_model')
filter = SelectKBest(score_func=chi2, k=200)
filter.fit(X,y)
print("RFEcv svc: ", cross_val(rfecv.transform(X)))
print("RFEcv log: ", cross_val(rfecv_log.transform(X)))
print("Filter: ", cross_val(filter.transform(X)))